
# CSD 튜토리얼


In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels as sm
import matplotlib
import matplotlib.pyplot as plt
import warnings
import time

In [2]:
import CSD

## 1. 계산 실행


기본적으로 하나의 연속된 일련의 공정 데이터를 분석하기 위해서는 하나의 CSD_Calculator class object를 사용한다.

CSD_Calculator 생성시에는 기본적으로
 1. csv_name : 분석하고자하는 파일명. string
 2. start_dates, final_dates : 분석하고자 하는 각 대상 공정들의 시작일/종료일의 칼럼명. list of string
 3. process_names : 분석하고자 하는 공정의 이름. list of string
 4. set_m : 각 공정이 보유하는 기계 대수. list of integer
    또는
    set_u : 전체 공정이 가지는 utilization 수치. double between 0 and 1 
 parameter를 명시해줘야 한다.

이외에도 추가적으로
 1. time_range : 분석하고자 하는 시간 범위. list of integer
 2. del_nan : 빈 칸이 있는 행을 삭제할 지 여부. boolean
 3. del_inconsistency : 한 공정의 종료일이 시작일보다 앞에 있는 행을 삭제할 지 여부. boolean
 parameter를 명시해줌으로써 전처리 옵션을 선택해 줄 수 있다. (기본값은 None, True, True)

In [3]:
csv_name = 'block_transfer_list_Raw_Data.csv'
SD = ['ASSY_ACTL_SD','OFT_ACTL_SD','PNT_ACTL_SD']
FD = ['ASSY_ACTL_FD','OFT_ACTL_FD','PNT_ACTL_FD']
process_names = ['ASSY_ACTL','OFT_ACTL','PNT_ACTL']
set_m = [300, 270, 240]

In [4]:
temp = time.time()
csd = CSD.CSD_Calculator(csv_name, start_dates = SD, final_dates = FD, set_m = set_m, set_u = None,
                 process_names = process_names, time_range = [20150101, 20180101], del_nan = True, del_inconsistency = True)
print(time.time() - temp)

11.36531949043274



위와 같이 필요한 정보를 입력하고 CSD_Calculator 객체를 생성해주면 자동으로 전처리 및 필요한 계수 계산을 진행한다.

계산 결과는 객체 내부에 DataFrame 형태로 저장되며, 다음과 같이 호출할 수 있다.


## 2. 결과 확인

### 1) 기본 계수

도착률(Ra), 도착간 시간 변동성계수(Ca),

작업시간(Te), 작업시간 변동성 계수(Ce),

출발률(Rd), 출발간시간 변동성 계수(Cd),

기계 대수(m), 가동률(u)

등 각 공정별 기본적인 변수들은 다음과 같이 클래스 내부에 var_table이란 이름의 DataFrame으로 저장된다.

In [5]:
csd.var_table

,ASSY_ACTL,OFT_ACTL,PNT_ACTL
Ra,16.4858,16.4858,16.4858
Ca,4.13665,7.204,5.24745
Te,16.2346,15.6568,12.7662
Ce,1.12154,1.23115,0.506727
Rd,15.1443,14.3612,15.4159
Cd,5.81914,5.26195,4.94709
m,300,270,240
u,0.892136,0.955983,0.876923


### 2) 주요 결과 값

CTq, CT,

WIPq, WIP

등 각 공정별 주요 결과 지표들은 다음과 같이 클래스 내부에 result_table이란 이름의 DataFrame으로 저장된다.

In [6]:
csd.result_table

,ASSY_ACTL,OFT_ACTL,PNT_ACTL
CTq,0.313965,1.40513,0.0216376
CT,16.5486,17.0619,12.7878
WIPq,5.17597,23.1647,0.356714
WIP,272.817,281.28,210.818


### 3) 기타 중간 산출 데이터

주요 변수 외에도, 작업 날짜 기록, 각 공정별 작업시간, 각 공정별 도착간 시간 간격을 내부에 저장하여 호출할 수 있다.


#### 전처리 완료된 공정별 시작/종료 날짜 테이블

클래스 내부에 df_dates라는 이름의 DataFrame으로 저장

In [7]:
csd.df_dates.head()

,ASSY_ACTL_SD_S,OFT_ACTL_SD_S,PNT_ACTL_SD_S,ASSY_ACTL_FD_F,OFT_ACTL_FD_F,PNT_ACTL_FD_F
17914,2015-01-21,2015-02-05,2015-02-26,2015-02-03,2015-02-13,2015-03-09
17981,2015-01-03,2015-01-27,2015-02-25,2015-01-30,2015-02-23,2015-03-09
18032,2015-01-06,2015-02-04,2015-02-25,2015-01-31,2015-02-23,2015-03-10
18092,2015-01-03,2015-01-27,2015-03-06,2015-01-30,2015-02-23,2015-03-18
18102,2015-01-05,2015-01-30,2015-02-25,2015-02-07,2015-02-23,2015-03-09


In [8]:
csd.df_dates.describe()

,ASSY_ACTL_SD_S,OFT_ACTL_SD_S,PNT_ACTL_SD_S,ASSY_ACTL_FD_F,OFT_ACTL_FD_F,PNT_ACTL_FD_F
count,18053,18053,18053,18053,18053,18053
unique,1054,1067,894,829,1000,925
top,2016-09-28 00:00:00,2015-12-14 00:00:00,2016-02-23 00:00:00,2015-08-31 00:00:00,2015-11-30 00:00:00,2016-02-29 00:00:00
freq,84,61,53,101,76,71
first,2015-01-01 00:00:00,2014-12-15 00:00:00,2015-01-11 00:00:00,2015-01-07 00:00:00,2015-01-05 00:00:00,2015-02-02 00:00:00
last,2017-12-31 00:00:00,2018-06-15 00:00:00,2018-03-29 00:00:00,2018-04-13 00:00:00,2018-06-15 00:00:00,2018-04-18 00:00:00


#### 공정 별 작업 시간 테이블

클래스 내부에 df_te라는 이름의 DataFrame으로 저장

In [9]:
csd.df_te.head()

,ASSY_ACTL,OFT_ACTL,PNT_ACTL
17914,14.0,9.0,12.0
17981,28.0,28.0,13.0
18032,26.0,20.0,14.0
18092,28.0,28.0,13.0
18102,34.0,25.0,13.0


#### 공정 별 도착(출발)간 시간 간격 테이블

클래스 내부에 iat라는 이름의 Numpy ndarray로 저장

In [10]:
csd.iat

array([[1., 3., 0., 0., 5., 0.],
       [0., 6., 0., 5., 1., 2.],
       [0., 1., 9., 0., 2., 0.],
       ...,
       [0., 0., 1., 4., 0., 0.],
       [1., 0., 0., 0., 0., 4.],
       [0., 0., 5., 3., 0., 2.]])

### ## 원하는 utilization 값 설정

단순히 각 공정 별 기계 대수를 입력해주는 게 아니라, 원하는 가동률(utilization)에 해당하는 기계값을 자동으로 설정해주도록 할 수 있다.
CSD_Calculator 클래스를 생성할 때 파라미터 set_m = None, set_u = (원하는 가동률 수준)을 설정해주면 된다.
실행 예시는 다음과 같다.

In [11]:
csv_name = 'block_transfer_list_Raw_Data.csv'
SD = ['ASSY_ACTL_SD','OFT_ACTL_SD','PNT_ACTL_SD']
FD = ['ASSY_ACTL_FD','OFT_ACTL_FD','PNT_ACTL_FD']
process_names = ['ASSY_ACTL','OFT_ACTL','PNT_ACTL']
set_u = 0.97

In [12]:
### 원하는 utilization을 맞춰주는 기능
temp = time.time()
csd = CSD.CSD_Calculator(csv_name, start_dates = SD, final_dates = FD, set_m = None, set_u = set_u,
                 process_names = process_names, time_range = [20150101, 20180101], del_nan = True, del_inconsistency = True)
print(time.time() - temp)

9.388554811477661


다음과 같이 utilization이 0.97이 되는 정수 m으로 맞춰진 것을 확인할 수 있다.

In [13]:
csd.var_table

,ASSY_ACTL,OFT_ACTL,PNT_ACTL
Ra,16.4858,16.4858,16.4858
Ca,4.13665,7.204,5.24745
Te,16.2346,15.6568,12.7662
Ce,1.12154,1.23115,0.506727
Rd,15.1443,14.3612,15.4159
Cd,5.81914,5.26195,4.94709
m,276,266,217
u,0.969713,0.970359,0.969868


## 3. 분포 함수 피팅

분포 함수 피팅을 위해서는 CSD 파일 내의 distribution_finder 함수를 이용하면 된다.

이 때 CSD_Calculator 클래스와는 분리되어 있으므로, 피팅을 위해서 꼭 CSD_Calculator를 통한 처리를 거칠 필요 없이 원하는 어떤 데이터든지 피팅이 가능하다.

인풋 데이터로는 Numpy ndarray, Pandas DataFrame, Pandas Series 모두 가능하게 설정해두었으며, 한 열의 데이터 당 하나의 분포함수를 피팅하도록 설정해두었다.



distribution_finder 사용을 위해서는 다음 parameter들을 명시해줘야한다.

1. table : 분포 함수 피팅을 하고자 하는 데이터. np.ndarray, pd.DataFrame, pd.Series
2. quantile : 데이터 전처리 옵션. 해당 분위수 이상의 데이터를 삭제함. default = None. double between 0 and 1
3. factor : 최적 분포함수 선정에 사용하는 지표. default = 'sse'. 'loglh' 선택 가능
4. tunning : 일차적으로 피팅한 분포함수를 원래 데이터의 평균과 같은 평균을 가지도록 scale 파라미터를 조정할 지 여부를 결정. default = True. boolean
5. accuracy : 분포함수 튜닝을 사용할 시, 원래 데이터의 평균과 얼마나 작은 오차의 평균을 가지도록 조정할 지 정해주는 값. default = 0.01. double
 * Inter Arrival Time을 피팅할 시, 0.0001 이하의 극히 작은 값 사용할 것 권장
6. _round : 분포함수 튜닝을 사용할 시, 분포 함수가 발생시킨 값을 반올림해서 사용할 지 여부를 결정. 원래 데이터가 정수로 표현되어 있으면 True를 사용할 것 권장. default = True. boolean
 * 또한 기본적으로 튜닝시에는 nonnegative한 값만을 사용하도록 설정되어 있다.

Output으로는 최종적으로 피팅된 Scipy.stats의 함수의 리스트, 파라미터의 리스트를 반환한다.
또한 분포함수 이름과 파라미터를 출력해주고, 원래 데이터와 피팅 함수로 발생시킨 난수의 주요지표를 비교하는 항목도 출력해준다.

 * 자세한 사항은 다음 실행 예시 참고


실행 예시는 다음과 같다.
먼저, 앞서 CSD_Calculator를 통해 계산한 데이터의 작업시간에 대한 분포를 피팅해보자.

In [14]:
dist_list_te, params_list_te = CSD.distribution_finder(csd.df_te, quantile = 0.99, accuracy = 0.01, _round = True)

Distribution 0 is tunning...
Tunned error : 0.004808870280841759
Distribution 1 is tunning...
Tunned error : 0.16979925769859783
Distribution 2 is tunning...
Tunned error : 0.00280024439448745



Found distributions are :
0 : exponnorm(K=5.13, loc=5.50, scale=1.86)
1 : pareto(b=5.16, loc=-61.21, scale=56.90)
2 : exponnorm(K=1.25, loc=8.66, scale=2.96)


Comparing original data with fitted distribution
0 : 
           Original        Fitting
count  17875.000000  999934.000000
mean      15.025287      15.029896
std       10.599488       9.714085
min        1.000000       0.000000
25%        8.000000       8.000000
50%       12.000000      12.000000
75%       18.000000      19.000000
max       75.000000     139.000000

CV
Original : 0.71    Fitting : 0.65



1 : 
           Original        Fitting
count  17876.000000  686004.000000
mean      14.495637      14.703442
std       14.216422      18.833585
min        1.000000       0.000000
25%        4.000000       4.000000
50%        9.000000

분포함수 출력, 원래 데이터와 피팅함수 난수 발생 데이터 비교 분석 결과 출력이 잘 된 것을 확인할 수 있다.


※ 참고 : 다음과 같이 생성된 함수와 파라미터의 구체적인 값의 확인 및 접근이 가능하다.

In [15]:
for dist in dist_list_te:
    print(dist)
print('\n')

for params in params_list_te:
    print(params)



(5.126511134808928, 5.501172958260842, 1.8578156422458711)
(5.164688646549719, -61.21309842264902, 56.897798202960416)
(1.2500416914409835, 8.662929596117616, 2.9642987442454904)


이렇게 피팅한 함수를 이용해 직접 난수를 발생시키고 싶다면 아래 코드를 참고하면 된다.

In [16]:
dist = dist_list_te[0] #위의 distribution_finder 함수에서 반환한 분포함수들의 리스트에서 원하는 함수 선택
params = params_list_te[0] #위의 distribution_finder 함수에서 반환한 파라미터들의 리스트에서 원하는 파라미터 선택

#이 부분은 그냥 따라하면 된다
arg = params[:-2]
loc = params[-2]
scale = params[-1]
#선택한 분포에 따른 난수 생성
a = dist.rvs(arg, loc, scale, size = 1000000)
a = a[a > 0] # 처리 옵션 : nonnegative
a = np.round(a) # 처리 옵션 : 반올림

#데이터 타입을 pd.Series로 변환하여 난수가 제대로 발생됐는지 확인
a = pd.Series(a)
print("CV : " + str(round(a.std() / a.mean(),2)))
a.describe()

CV : 0.65


count    999911.000000
mean         15.043199
std           9.709815
min           0.000000
25%           8.000000
50%          12.000000
75%          19.000000
max         120.000000
dtype: float64

도착간 시간 간격(inter arrival time)을 피팅한 결과 예시는 다음과 같다.

※예시 데이터의 경우 극단값이 없기 때문에 quantile = None 사용

※Inter Arrival Time을 피팅할 시, 평균의 작은 차이가 도착률(Ra)에 큰 영향을 미쳐 시뮬레이션 값이 큰 영향을 미치므로, 보다 정확한 피팅을 위해 accuracy = 0.0001 이하의 극히 작은 값 사용할 것 권장

In [17]:
dist_list_iat, params_list_iat = CSD.distribution_finder(csd.iat, quantile = None, accuracy = 0.001, _round = True)

Distribution 0 is tunning...
Tunned error : 0.00021609882561488686
Distribution 1 is tunning...
Tunned error : 0.0004614797252382036
Distribution 2 is tunning...
Tunned error : 0.00029194970086417193
Distribution 3 is tunning...
Tunned error : 0.00033053534234434023
Distribution 4 is tunning...
Tunned error : 0.0004668262796365952
Distribution 5 is tunning...
Tunned error : 0.0006158413472191482



Found distributions are :
0 : chi2(df=1.53, loc=-0.00, scale=0.11)
1 : chi2(df=1.53, loc=-0.00, scale=0.11)
2 : chi2(df=1.53, loc=-0.00, scale=0.11)
3 : chi2(df=1.53, loc=-0.00, scale=0.11)
4 : chi2(df=1.53, loc=-0.00, scale=0.11)
5 : chi2(df=1.53, loc=-0.00, scale=0.11)


Comparing original data with fitted distribution
0 : 
           Original         Fitting
count  18052.000000  1000000.000000
mean       0.060658        0.060150
std        0.250928        0.239629
min        0.000000        0.000000
25%        0.000000        0.000000
50%        0.000000        0.000000
75%        0.00000

마찬가지로, 피팅한 함수를 이용해 직접 난수를 발생시키고 싶다면 아래 코드를 참고하면 된다.
distribution_finder 함수에서 반환한 분포와 파라미터 리스트 중 원하는 분포와 리스트만 선택해 아래 코드를 그대로 사용하면 된다.

In [18]:
dist = dist_list_iat[0] #위의 distribution_finder 함수에서 반환한 분포함수들의 리스트에서 원하는 함수 선택
params = params_list_iat[0] #위의 distribution_finder 함수에서 반환한 파라미터들의 리스트에서 원하는 파라미터 선택

#이 부분은 그냥 따라하면 된다
arg = params[:-2]
loc = params[-2]
scale = params[-1]
#선택한 분포에 따른 난수 생성
a = dist.rvs(arg, loc, scale, size = 1000000)
a = a[a > 0] # 처리 옵션 : nonnegative
a = np.round(a) # 처리 옵션 : 반올림

#데이터 타입을 pd.Series로 변환하여 난수가 제대로 발생됐는지 확인
a = pd.Series(a)
print("CV : " + str(round(a.std() / a.mean(),2)))
a.describe()

CV : 3.97


count    1000000.000000
mean           0.060710
std            0.240766
min            0.000000
25%            0.000000
50%            0.000000
75%            0.000000
max            3.000000
dtype: float64

### 부록 - cal_result

CSD안에는 cal_result 라는 함수가 존재한다. 이를 통해 한 공정의 기본 변수를 입력하기만 하면 해당 공정의 CT, WIP를 자동으로 계산해 반환해준다.
비슷하게 여러개의 연속된 공정의 기본 변수를 입력하면 각 공정의 CT, WIP를 계산해주는 cal_result_table 이라는 함수도 존재한다. 각 함수의 input, output은 다음과 같다.

1. cal_result
input : te, ce, ra, ca, u, m
output : CTq, CT, WIPq, WIP

2. cal_result_table
input : var_table, process_names
output : result_table